In [ ]:
%pip install gymnasium
%pip install sklearn
%pip install collections
%pip install keras
%pip install matplotlib
%pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.1 MB/s eta 0:00:00


# Новый раздел

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
import random
from collections import deque
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn import preprocessing
from sklearn import utils
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
import catboost as cb
from catboost import CatBoostClassifier
from sklearn import datasets

The function representing the value
of an action can be seen as a table that maps all states
and all actions to the expected long-term return. In our
case, the dimension of this table is large and compiling it
requires high computational costs.

In [ ]:
def normalize(data):
    mean = data.mean(axis=0)
    std = data.std(axis=0)
    data -= mean
    data /= std
    return data

In [ ]:
class Detector:
    def __init__(self, classifier, class_column_name):
        self.classifier = classifier
        self.class_column_name = class_column_name

    def predict(self, x):
        y = self.classifier.predict(x);
        return y

    def test(self, data):
        y = data[self.class_column_name]
        x = data.drop(self.class_column_name, axis=1)
        x = normalize(x)
        y_pred = self.classifier.predict(x)
        accuracy = accuracy_score(y, y_pred)
        matrix = confusion_matrix(y, y_pred)
        report = classification_report(y, y_pred)
        print("Accuracy:", accuracy)
        print(matrix)
        print(report)

    def fit(self, data):
        y = data[self.class_column_name]
        X = data.drop(self.class_column_name, axis=1)
        X = normalize(X)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        train_pool = cb.Pool(X_train, y_train)
        val_pool = cb.Pool(X_val, y_val)
        self.classifier.fit(train_pool, eval_set=val_pool, early_stopping_rounds=100)

In [ ]:
class Enviroment(gym.Env):
    def __init__(self, detector, attack_traffic, actions, class_column_name, attack_class):
        super().__init__()
        self.result = []
        self.class_column_name = class_column_name
        self.attack_class = attack_class
        # self.result = pd.DataFrame({"Flow Duration": [], "TotLen Fwd Pkts": [], "TotLen Bwd Pkts": [],
        #                                      "Flow Byts/s": [], "Flow Pkts/s": [], "Bwd/Fwd Ratio": [], "Pkt Size Avg": [], "Malware": []})
        self.detector = detector
        self.current_state_id = 0
        self.start_states = attack_traffic
        self.start = attack_traffic.iloc[self.current_state_id]
        self.state = attack_traffic.iloc[self.current_state_id]
        self.space = []
        for action in actions:
          self.space.append({"Name": action, "Action": "+" + action, "Value": 0.01})
          self.space.append({"Name": action, "Action": "-" + action, "Value": -0.01})

    def reset(self):
        self.current_state_id = self.current_state_id + 1 if self.current_state_id + 1 < len(self.start_states) else 0
        self.start_states.iloc[self.current_state_id]
        return self.state.values.reshape(1, -1)

    def step(self, action_n):
        action = self.space[action_n]
        self.state[action["Name"]] += action["Value"]
        result = self.detector.predict(self.state.values.reshape(1, -1))
        if result == 0:
          curr_state = self.state.copy()
          curr_state[self.class_column_name] = self.attack_class
          self.result.append(curr_state.to_dict())

          # self.result = pd.concat([self.result, curr_state], ignore_index=True)
          self.current_state_id = self.current_state_id + 1 if self.current_state_id + 1 < len(self.start_states) else 0
          self.state = self.start_states.iloc[self.current_state_id]
        return (self.state.values.reshape(1, -1), 1 if result == 0 else 0, True, True if result == 0 else False)

In [ ]:
class DQLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size # Сколько фич изменяем
        self.action_size = action_size # В каком объеме меняем
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.model = self.build_model()
        self.mean_losses = []
        self.mean_rewards = []

    def build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        opt = keras.optimizers.Adam(use_ema=True)
        model.compile(loss='mse', optimizer=opt)
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

            mean_loss = np.mean(np.square(target_f - self.model.predict(state)))
            mean_reward = np.mean([reward for _, _, reward, _, _ in minibatch])
            self.mean_losses.append(mean_loss)
            self.mean_rewards.append(mean_reward)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
class Learner:
  def __init__(self, detector, data, actions, attack_class, class_column_name):
    self.agent = DQLAgent(len(actions), len(actions)*2)
    states = data.loc[data[class_column_name] == attack_class]
    states = states.drop(class_column_name, axis=1)
    states = normalize(states)
    self.env = Enviroment(detector, states, actions, class_column_name, attack_class)

  def fit(self, epoch_n):
    ls = []
    for _ in range(epoch_n):
      state = self.env.reset()
      action = self.agent.act(state)
      new_state, reward, done, sucess = self.env.step(action)
      if not sucess:
        reward = np.amax(self.agent.model.predict(new_state)[0])
      self.agent.remember(state, action, reward, new_state, done)
      k = 1
      while not sucess:
        k += 1
        action = self.agent.act(state)
        new_state, reward, done, sucess = self.env.step(action)
        if not sucess:
          reward = np.amax(self.agent.model.predict(new_state)[0])
        self.agent.remember(state, action, reward, new_state, done)
      self.agent.replay(25)
      ls.append(k)
    self.agent.model.save('model.h5')
    print(sum(ls) / len(ls))

  def show_mean_loss(self):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(len(self.agent.mean_losses)), self.agent.mean_losses)
    plt.xlabel('Iterations')
    plt.ylabel('Mean Loss')
    plt.title('Mean Loss over Iterations')
    plt.subplot(1, 2, 2)
    plt.plot(range(len(self.agent.mean_rewards)), self.agent.mean_rewards)
    plt.xlabel('Iterations')
    plt.ylabel('Mean Reward')
    plt.title('Mean Reward over Iterations')
    plt.tight_layout()
    plt.show()

Классификатор на букинг датасете

In [ ]:
data_booking = pd.read_csv('booking.csv')
data_booking, test_data_booking = train_test_split(data_booking, test_size=0.2, random_state=42)
classifier_booking = CatBoostClassifier(iterations=10000, depth=8, learning_rate=0.08)
detector_booking = Detector(classifier_booking, 'booking status')
detector_booking.fit(data_booking)
detector_booking.test(test_data_booking)

0:	learn: 0.6441490	test: 0.6442956	best: 0.6442956 (0)	total: 10.5ms	remaining: 1m 44s
1:	learn: 0.6051035	test: 0.6051380	best: 0.6051380 (1)	total: 20ms	remaining: 1m 39s
2:	learn: 0.5734847	test: 0.5737255	best: 0.5737255 (2)	total: 31.3ms	remaining: 1m 44s
3:	learn: 0.5484314	test: 0.5490972	best: 0.5490972 (3)	total: 40.8ms	remaining: 1m 41s
4:	learn: 0.5253379	test: 0.5260559	best: 0.5260559 (4)	total: 50ms	remaining: 1m 40s
5:	learn: 0.5075203	test: 0.5086371	best: 0.5086371 (5)	total: 58.8ms	remaining: 1m 37s
6:	learn: 0.4936029	test: 0.4948067	best: 0.4948067 (6)	total: 68.3ms	remaining: 1m 37s
7:	learn: 0.4796838	test: 0.4812860	best: 0.4812860 (7)	total: 77.5ms	remaining: 1m 36s
8:	learn: 0.4688776	test: 0.4708158	best: 0.4708158 (8)	total: 87.2ms	remaining: 1m 36s
9:	learn: 0.4602753	test: 0.4624997	best: 0.4624997 (9)	total: 96.9ms	remaining: 1m 36s
10:	learn: 0.4510316	test: 0.4534711	best: 0.4534711 (10)	total: 106ms	remaining: 1m 36s
11:	learn: 0.4445623	test: 0.447137

Классификатор на ботнет датасете

In [ ]:
data = pd.read_csv('Botnet-Training-Short.csv')
classifier = CatBoostClassifier(iterations=10000, depth=8, learning_rate=0.08)
detector = Detector(classifier, 'Malware')
detector.fit(data)
test_data = pd.read_csv('Botnet-Testing-Short.csv')
detector.test(test_data)

0:	learn: 0.4767587	test: 0.4779323	best: 0.4779323 (0)	total: 19.2ms	remaining: 3m 12s
1:	learn: 0.3402008	test: 0.3419548	best: 0.3419548 (1)	total: 35.5ms	remaining: 2m 57s
2:	learn: 0.2571400	test: 0.2590844	best: 0.2590844 (2)	total: 51.9ms	remaining: 2m 52s
3:	learn: 0.1996325	test: 0.2019509	best: 0.2019509 (3)	total: 68.2ms	remaining: 2m 50s
4:	learn: 0.1645643	test: 0.1669014	best: 0.1669014 (4)	total: 86.2ms	remaining: 2m 52s
5:	learn: 0.1350855	test: 0.1375237	best: 0.1375237 (5)	total: 105ms	remaining: 2m 54s
6:	learn: 0.1139691	test: 0.1166254	best: 0.1166254 (6)	total: 121ms	remaining: 2m 53s
7:	learn: 0.1007244	test: 0.1032899	best: 0.1032899 (7)	total: 140ms	remaining: 2m 55s
8:	learn: 0.0888382	test: 0.0914309	best: 0.0914309 (8)	total: 158ms	remaining: 2m 55s
9:	learn: 0.0816006	test: 0.0842063	best: 0.0842063 (9)	total: 176ms	remaining: 2m 56s
10:	learn: 0.0746324	test: 0.0771646	best: 0.0771646 (10)	total: 195ms	remaining: 2m 56s
11:	learn: 0.0696120	test: 0.0722042

Агент на букинг классификаторе

In [ ]:
data_booking_ = pd.read_csv('booking.csv')
data_booking_, test_data_booking_ = train_test_split(data_booking_, test_size=0.2, random_state=42)
learner_booking = Learner(detector_booking, data_booking_,
                  ['number of adults','number of children','number of weekend nights','number of week nights',
                   'car parking space','lead time','repeated','P-C','P-not-C','average price','special requests'],
                  1, 'booking status')
learner_booking.fit(5)
learner_booking.show_mean_loss()

1/1 [==============================] - ETA: 0s

Агент на ботнет классификаторе

In [ ]:
df = pd.read_csv('Botnet-Training-Short.csv')
learner = Learner(detector, df,
 ["Flow Duration", "TotLen Fwd Pkts", "TotLen Bwd Pkts", "Flow Byts/s", "Flow Pkts/s", "Bwd/Fwd Ratio", "Pkt Size Avg"],
                  1, 'Malware')
learner.fit(100)
learner.show_mean_loss()

CatBoostError: /src/catboost/catboost/libs/data/model_dataset_compatibility.cpp:72: Feature P-C is present in model but not in pool.

In [ ]:
model = load_model('model.h5')
agent.model = model
learner = Learner(agent, env)

In [ ]:
data_new = pd.read_csv('Botnet-Training-Short.csv')
print(len(data_new))
data_new = pd.concat([data_new, pd.DataFrame(learner.env.result)], ignore_index=True)
print(len(data_new))
classifier_new = CatBoostClassifier(iterations=10000, depth=8, learning_rate=0.08)
detector_new = Detector(classifier_new, 'Malware')
detector_new.fit(data_new)
test_data = pd.read_csv('Botnet-Testing-Short.csv')
detector_new.test(test_data)

56961
57061
0:	learn: 0.4889099	test: 0.4908385	best: 0.4908385 (0)	total: 15.6ms	remaining: 2m 35s
1:	learn: 0.3395662	test: 0.3421552	best: 0.3421552 (1)	total: 31.3ms	remaining: 2m 36s
2:	learn: 0.2572173	test: 0.2601094	best: 0.2601094 (2)	total: 46.9ms	remaining: 2m 36s
3:	learn: 0.1990808	test: 0.2022768	best: 0.2022768 (3)	total: 62.2ms	remaining: 2m 35s
4:	learn: 0.1554449	test: 0.1584495	best: 0.1584495 (4)	total: 78ms	remaining: 2m 36s
5:	learn: 0.1325394	test: 0.1352961	best: 0.1352961 (5)	total: 92.3ms	remaining: 2m 33s
6:	learn: 0.1139345	test: 0.1165980	best: 0.1165980 (6)	total: 109ms	remaining: 2m 36s
7:	learn: 0.1025628	test: 0.1052830	best: 0.1052830 (7)	total: 125ms	remaining: 2m 36s
8:	learn: 0.0922027	test: 0.0948896	best: 0.0948896 (8)	total: 140ms	remaining: 2m 35s
9:	learn: 0.0842563	test: 0.0869612	best: 0.0869612 (9)	total: 159ms	remaining: 2m 38s
10:	learn: 0.0791313	test: 0.0819021	best: 0.0819021 (10)	total: 174ms	remaining: 2m 38s
11:	learn: 0.0727078	test

        self.space = [{"Name": "Flow Duration", "Action": "+Flow Duration", "Value": 0.01},
                      {"Name": "Flow Duration", "Action": "-Flow Duration", "Value": -0.01},
                      {"Name": "TotLen Fwd Pkts", "Action": "+TotLen Fwd Pkts", "Value": 0.01},
                      {"Name": "TotLen Fwd Pkts", "Action": "-TotLen Fwd Pkts", "Value": -0.01},
                      {"Name": "TotLen Bwd Pkts", "Action": "+TotLen Bwd Pkts", "Value": 0.01},
                      {"Name": "TotLen Bwd Pkts", "Action": "-TotLen Bwd Pkts", "Value": -0.01},
                      {"Name": "Flow Byts/s", "Action": "+Flow Byts/s", "Value": 0.01},
                      {"Name": "Flow Byts/s", "Action": "-Flow Byts/s", "Value": -0.01},
                      {"Name": "Flow Pkts/s", "Action": "+Flow Pkts/s", "Value": 0.01},
                      {"Name": "Flow Pkts/s", "Action": "-Flow Pkts/s", "Value": -0.01},
                      {"Name": "Bwd/Fwd Ratio", "Action": "+Bwd/Fwd Ratio", "Value": 0.01},
                      {"Name": "Bwd/Fwd Ratio", "Action": "-Bwd/Fwd Ratio", "Value": -0.01},
                      {"Name": "Pkt Size Avg", "Action": "+Pkt Size Avg", "Value": 0.01},
                      {"Name": "Pkt Size Avg", "Action": "-Pkt Size Avg", "Value": -0.01}]

In [ ]:
# class CEM():
#     def __init__(self, state_n, action_n):
#       self.state_n = state_n
#       self.action_n = action_n
#       self.policy = np.ones((self.state_n, self.action_n)) / self.action_n

#     def get_action(self, state):
#       return int(np.random.choice(np.arange(self.action_n), p=self.policy[state]))

#     def update_policy(self, elite_tr):
#       pre_policy = np.zeros((state_n, action_n))

#       for tr in elite_tr:
#         for state, action in zip(tr['states'], tr['actions']):
#           pre_policy[state][action] += 1

#           for state in range(self.action_n):
#             if sum(pre_policy[state]) == 0:
#               self.policy[state] = np.ones(self.action_n) / self.action_n
#             else:
#               self.policy[state] = pre_policy[state] / sum(pre_policy[state])

In [ ]:
# class CEMAgent():
#     def __init__(self, env, agent_logic):
#       self.env = env
#       self.agent_logic = agent_logic

#     def get_state(self):
#       return None

#     def get_trajectory(self, tr_len):
#       tr = {'states': [], 'actions': [], 'total_reward': 0}

#       obs = self.env.reset()
#       state = get_state(obs)
#       tr['states'].append(state)

#       for _ in range(tr_len):
#           action = self.agent_logic.get_action(state)
#           tr['actions'].append(action)

#           obs, reward, done = env.step(action)
#           state = get_state(obs)
#           tr['total_reward'] += reward

#           if done:
#               break

#           tr['states'].append(state)

#         return tr
#       def get_elite_trajectories(self, trajectories, q):
#           total = [tr['total_reward'] for tr in trajectories]
#           quantile = np.quantile(total, q=q)
#           return [tr for tr in trajectories if tr['total_reward'] > quantile]

#       def fit(self, episode_n, trajectory_n, trajectory_len, q):
#           for _ in range(episode_n):
#               trajectories = [get_trajectory(trajectory_len) for _ in range(trajectory_n)]

#               mean_total = np.mean([trajectory['total_reward'] for trajectory in trajectories])
#               print(mean_total)
#               elite = get_elite_trajectories(trajectories, q)

#               if len(elite) > 0:
#                   self.agent_logic.update_policy(elite)